In [1]:
from characters_real_names import *

import re
import spacy
from spacy.tokens import Span

nlp = spacy.load("en_core_web_sm")

# Used to get the gender and distinct people with same family noun. 

def get_person_title(span):
    if span.label_ == "PERSON" and span.start != 0:
        prev_token = span.doc[span.start - 1]
        if prev_token.text in ("Mr", "Mr.", "Mrs", "Mrs."):
            return prev_token.text + " "
    return ""

Span.set_extension("person_title", getter=get_person_title, force=True)

book = open("../Homework 1/Harry Potter and the Sorcerer.txt").read()

book_chapters = re.split(r'CHAPTER [\w+]+', book, flags=re.IGNORECASE)[1:]
book_chapters[-1] = book_chapters[-1].split("THE END")[0]


In [2]:
docs = nlp(''.join(book_chapters))
all_sentences = [sent.text.strip() for sent in docs.sents]

In [3]:
from tqdm import tqdm

# 7267 iterations
for i,sentence in tqdm(enumerate(all_sentences)):
    doc = nlp(sentence)
    entities = [dict(title=ent._.person_title,name=ent.text,position=(ent.start,ent.end)) for ent in doc.ents if ent.label_ == "PERSON"]

    for entity in entities:

        if entity.get('name') in all_real_names.keys(): continue
        for real,check in all_real_names.items():
            if entity.get('title') + entity.get('name') in check: 
                all_sentences[i] = sentence.replace(entity.get('name'), real)

7267it [00:55, 130.19it/s]


In [4]:
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags     
from gensim.parsing.preprocessing import strip_short      
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_punctuation, strip_non_alphanum

# select the filters you wish to apply here
CUSTOM_FILTERS = [lambda x: x.lower(), strip_non_alphanum, remove_stopwords, strip_multiple_whitespaces]

all_sentences_preprocessed = []
for sent in tqdm(all_sentences):
    parsed_line = preprocess_string(sent, CUSTOM_FILTERS)
    all_sentences_preprocessed.append(parsed_line)

100%|██████████| 7267/7267 [00:00<00:00, 53973.57it/s]


In [5]:
all_sentences_preprocessed[0]

['mr',
 'mrs',
 'petunia',
 'dursley',
 'number',
 'privet',
 'drive',
 'proud',
 'perfectly',
 'normal',
 'thank']